### Yolo11 tests

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os , sys
import shutil
from tqdm import tqdm  
sys.path.append('..')
from pathlib import Path
cur_path = Path(os.getcwd()).parent
sam2_path = cur_path / 'sam2_octron'
sys.path.append(cur_path.as_posix())
from matplotlib import pyplot as plt
import cmasher as cmr
import numpy as np
import seaborn as sns
sns.set_theme(style='white')
%config InlineBackend.figure_format = 'retina'

In [3]:
# Object organizer
# #### Importing additional stuff 
# from skimage import measure
# from skimage.draw import polygon2mask

In [4]:
from napari_pyav._reader import FastVideoReader
from octron.sam2_octron.helpers.video_loader import probe_video
from octron.yolo_octron.helpers.training import (load_object_organizer, 
                                                 collect_labels, 
                                                 draw_polygons,
                                                 train_test_val,
                                                 write_training_data,
                                                 write_yolo_config_yaml,
                                                 
)

In [5]:

from ultralytics import YOLO



In [6]:
# Presaved model (YOLOv11)
path_to_model = Path('/Users/horst/Documents/python/OCTRON/octron/yolo_octron/yolo11l-seg.pt')
# Output data from annotation in OCTRON
path_to_json_organizer = Path('/Users/horst/Downloads/octron_project/object_organizer.json')
# Video
path_to_video = Path('/Users/horst/Downloads/octron_project/first_half.mp4')

# Training data path 
path_to_training_root = path_to_json_organizer.parent / 'train_data'
path_to_training_root.mkdir(exist_ok=True)

# Video reader 
video_dict = probe_video(path_to_video)
video_data = FastVideoReader(path_to_video)  

File: /Users/horst/Downloads/octron_project/first_half.mp4
Codec: h264
Resolution: 1000 x 1000
Frame Rate: 7
Number of frames: 2036
Duration: 290.86 seconds


### Process training data

In [7]:
organizer_dict = load_object_organizer(path_to_json_organizer)  
assert organizer_dict is not None
labels = collect_labels(organizer_dict,
                        expected_num_frames=video_dict['num_frames'],
                        expected_image_height=video_dict['height'],
                        expected_image_width=video_dict['width'],
                        )

📖 Octron object organizer loaded from /Users/horst/Downloads/octron_project/object_organizer.json
Label wormsy has 263 annotated frames
Label handle has 263 annotated frames


Polygons for label handle: 100%|██████████| 263/263 [00:05<00:00, 52.42it/s]


In [8]:
#draw_polygons(labels, video_data, 2)

In [9]:
# Perform the split of the data
# .. 80% training, 10% validation and 10% testing
print('Splitting the data for training into training, validation and testing fractions')  
for label_id, label_dict in labels.items():
    print(label_dict['label'])
    label_split = train_test_val(label_dict['frames'], 
                                 training_fraction=0.8,
                                 validation_fraction=0.1,
                                 verbose=True)
    labels[label_id]['frames_split'] = label_split

Splitting the data for training into training, validation and testing fractions
wormsy
Total frames: 263
Training set: 210 frames
Validation set: 26 frames
Test set: 27 frames
handle
Total frames: 263
Training set: 210 frames
Validation set: 26 frames
Test set: 27 frames


In [10]:
# Now export the data to the respective directories
# We need to export the images and the labels

for split in ['train', 'val', 'test']:
    path_to_split = path_to_training_root / split
    try:
        path_to_split.mkdir(exist_ok=False)
    except FileExistsError:
        shutil.rmtree(path_to_split)    
        path_to_split.mkdir()

write_training_data(labels,
                    path_to_training_root,
                    video_data,
)


Exporting 2 labels: 100%|██████████| 2/2 [00:42<00:00, 21.43s/it]

Training data exported to /Users/horst/Downloads/octron_project/train_data


In [11]:
# Example usage
dataset_path = path_to_training_root
train_path = "train"  
val_path = "val"
test_path = "test"

# Get label names from your object organizer
label_id_label_dict = {}
for label_id, label_dict in labels.items():
    label_id_label_dict[label_id] = label_dict['label']

# Write the YAML config
config_path = path_to_training_root / "yolo_config.yaml"
write_yolo_config_yaml(
    output_path=config_path,
    dataset_path=dataset_path,
    train_path=train_path,
    val_path=val_path,
    test_path=test_path,
    label_dict=label_id_label_dict
)

YOLO config saved to /Users/horst/Downloads/octron_project/train_data/yolo_config.yaml


PosixPath('/Users/horst/Downloads/octron_project/train_data/yolo_config.yaml')

In [12]:
from ultralytics import settings
settings.update({'sync': False,'hub':False, })
runs_output_dir = config_path.parent / 'yolo runs'    
settings.update({'datasets_dir': '','weights_dir':'', 'runs_dir': runs_output_dir.as_posix(),
                 })

In [ ]:
# Load a model
model = YOLO(path_to_model)  # load a pretrained model (recommended for training)
# Train the model
# https://docs.ultralytics.com/usage/cfg/#solutions-settings
results = model.train(data=config_path, 
                      save_dir=runs_output_dir.as_posix(),
                      epochs=10,
                      imgsz=1000,
                      plots=True,
                      batch=.8,
                      cache=False,
                      save=True,
                      save_period=10,
                      project=None,
                      name=None,
                      exist_ok=True,
                      resume=False,
                      mask_ratio=2,
                      device='cpu',
                      # augmentation
                      hsv_v=.6,
                      degrees=180,
                      scale=.5,
                      shear=25,
                      flipud=.1,
                      fliplr=.1,
                      mosaic=1.0,
                      erasing=.25,
                      crop_fraction=1.0,
                      )

Ultralytics 8.3.81 🚀 Python-3.11.11 torch-2.5.1 CPU (Apple M2 Max)
engine/trainer: task=segment, mode=train, model=/Users/horst/Documents/python/OCTRON/octron/yolo_octron/yolo11l-seg.pt, data=/Users/horst/Downloads/octron_project/train_data/yolo_config.yaml, epochs=10, time=None, patience=100, batch=0.8, imgsz=1000, save=True, save_period=10, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=2, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=

train: Scanning /Users/horst/Downloads/octron_project/train_data/train... 210 images, 0 backgrounds, 0 corrupt: 100%|██████████| 210/210 [00:00<00:00, 2262.28it/s]

train: New cache created: /Users/horst/Downloads/octron_project/train_data/train.cache
AutoBatch: Computing optimal batch size for imgsz=1024 at 80.0% CUDA memory utilization.
AutoBatch:  ⚠️ intended for CUDA devices, using default batch-size 16



train: Scanning /Users/horst/Downloads/octron_project/train_data/train.cache... 210 images, 0 backgrounds, 0 corrupt: 100%|██████████| 210/210 [00:00<?, ?it/s]
val: Scanning /Users/horst/Downloads/octron_project/train_data/val... 26 images, 0 backgrounds, 0 corrupt: 100%|██████████| 26/26 [00:00<00:00, 2104.03it/s]

val: New cache created: /Users/horst/Downloads/octron_project/train_data/val.cache
Plotting labels to /Users/horst/Downloads/octron_project/train_data/yolo runs/segment/train/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 176 weight(decay=0.0), 187 weight(decay=0.0005), 186 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1024 train, 1024 val
Using 0 dataloader workers
Logging results to /Users/horst/Downloads/octron_project/train_data/yolo runs/segment/train
Starting training for 10 epochs...
Closing dataloader mosaic

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       1/10         0G      1.384      4.104      6.165      1.846         38       1024:   7%|▋         | 1/14 [01:31<19:49, 91.50s/it]

In [ ]:
#model = YOLO(path_to_model)  # load a pretrained model (recommended for training)


In [ ]:
# # Train/val/test sets as 
# 1) dir: path/to/imgs, 
# 2) file: path/to/imgs.txt, or list: [path/to/imgs1, path/to/imgs2, ..]
# path: ../datasets/coco8-seg # dataset root dir (absolute or relative; if relative, it's relative to default datasets_dir)
# train: images/train # train images (relative to 'path') 4 images
# val: images/val # val images (relative to 'path') 4 images
# test: # test images (optional)
